In [ ]:
import sys 
import os 
import numpy as np
import pandas as pd
import sklearn as sk
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn import metrics as metrics_sk
from sklearn.metrics import precision_recall_curve
from scipy.stats import ks_2samp
from IPython.display import Image, display_svg, SVG

In [ ]:
from keras import layers, Sequential
from keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding1D, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import metrics
from keras import optimizers
from keras import regularizers
import pydot
from tensorflow.keras import regularizers
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import tensorflow as tf
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflo

In [ ]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [ ]:
from keras.backend import log
def inv_h_sin(x, theta = 1):
    if x == 0:
        return x
    else:
        return log(theta*x + ((theta*x)**2 + 1)**0.5)/theta

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
get_custom_objects().update({'inv_h_sin': Activation(inv_h_sin)})

In [ ]:
os.getcwd()

'C:\\Users\\Benedito'

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
os.chdir("C:/Users/Benedito/Desktop/MECAI/Introdução a Estatistica/MVP")

In [ ]:
os.getcwd()

'C:\\Users\\Benedito\\Desktop\\MECAI\\Introdução a Estatistica\\MVP'

In [ ]:
df = pd.read_csv("modelling_table5.csv")

C:\Anaconda3\envs\benedito_deep\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (623,625,626,629) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data = df

In [ ]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [ ]:
data.dtypes.to_csv("modelling_5_dtypes.csv")

In [ ]:
categorical_feats = ['MODE(previous.CHANNEL_TYPE)',
'MODE(previous.CODE_REJECT_REASON)',
'MODE(previous.FLAG_LAST_APPL_PER_CONTRACT)',
'MODE(previous.NAME_CASH_LOAN_PURPOSE)',
'MODE(previous.NAME_CLIENT_TYPE)',
'MODE(previous.NAME_CONTRACT_STATUS)',
'MODE(previous.NAME_CONTRACT_TYPE)',
'MODE(previous.NAME_GOODS_CATEGORY)',
'MODE(previous.NAME_PAYMENT_TYPE)',
'MODE(previous.NAME_PORTFOLIO)',
'MODE(previous.NAME_PRODUCT_TYPE)',
'MODE(previous.NAME_SELLER_INDUSTRY)',
'MODE(previous.NAME_TYPE_SUITE)',
'MODE(previous.NAME_YIELD_GROUP)',
'MODE(previous.PRODUCT_COMBINATION)',
'MODE(previous.WEEKDAY_APPR_PROCESS_START)',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[0]',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[1]',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[2]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[0]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[1]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[2]',
'N_MOST_COMMON(previous.FLAG_LAST_APPL_PER_CONTRACT)[0]',
'N_MOST_COMMON(previous.FLAG_LAST_APPL_PER_CONTRACT)[1]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[0]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[1]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[2]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[0]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[1]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[2]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[0]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[1]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[2]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[0]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[1]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[2]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[0]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[1]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[2]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[0]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[1]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[2]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[0]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[1]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[2]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[0]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[1]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[2]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[0]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[1]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[2]',
'MODE(bureau.CREDIT_ACTIVE)',
'MODE(bureau.CREDIT_CURRENCY)',
'MODE(bureau.CREDIT_TYPE)',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[0]',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[1]',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[2]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[0]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[1]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[2]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[0]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[1]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[2]']

In [ ]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [ ]:
data[categorical_feats].fillna("no_info", inplace = True)

C:\Anaconda3\envs\benedito_deep\lib\site-packages\pandas\core\frame.py:4323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [ ]:
def check_string(x):
    if x in ('NaN', 'NA', 'n/a', 'N/A', 'inf', '-inf', 'Inf', '-Inf'):
        return True
    else:
        return False

In [ ]:
def fill_cat(df, lista_cat):
    for cat_feat in lista_cat:
        df.loc[pd.isna(df[cat_feat]), cat_feat] = 'no_info'
        df.loc[df[cat_feat].isnull(), cat_feat] = 'no_info'
        df.loc[df[cat_feat].apply(check_string), cat_feat] = 'no_info'
    return df
    

In [ ]:
data = fill_cat(data, categorical_feats)

In [ ]:
data.fillna(0, inplace = True)

In [ ]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [ ]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [ ]:
data = data.drop(categorical_feats, axis = 1)

In [ ]:
data.set_index('SK_ID_CURR', inplace=True)

In [ ]:
data = data.replace([np.inf, -np.inf], 0)

In [ ]:
data.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_start_dawn_previous,cnt

In [ ]:
#data = pd.get_dummies(data, columns = categorical_feats, drop_first = True)

In [ ]:
data.to_csv("modelling_table_6.csv")

In [ ]:
np.random.seed(11235)
sample = data.sample(200000, random_state = 11235)

In [ ]:
valid = data[~data.index.isin(sample.index)].sample(100000, random_state = 11235)
test_final = data[~((data.index.isin(valid.index))|data.index.isin(sample.index))]

In [ ]:
X = sample.drop(['TARGET'], axis =1)
y = sample['TARGET']

In [ ]:
X.fillna(0, inplace = True)

In [ ]:
#smote = SMOTE(sampling_strategy = 0.1, random_state = 11235) ##0.15 fica bom

In [ ]:
#X_, y_ = X, y

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3, random_state=1234)

In [ ]:
#from keras.callbacks import EarlyStopping

In [ ]:
#callback = EarlyStopping(monitor='AUC', patience=5)

## Neural Network

In [ ]:
### MELHOR DE TODAS
NN_Model = Sequential()
NN_Model.add(Dense(32, input_dim=X.shape[-1], activation='swish'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(256, activation='swish'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(32, activation='swish'))
NN_Model.add(Dropout(0.5, seed = 11235))
NN_Model.add(Dense(1, activation='sigmoid'))

In [ ]:
metrics_ = [metrics.FalseNegatives(name="fn"),metrics.FalsePositives(name="fp"),
           metrics.TrueNegatives(name="tn"),
           metrics.TruePositives(name="tp"),
           metrics.Precision(name="precision"),
           metrics.Recall(name="recall"),
           metrics.AUC(name="AUC"), 
           metrics.Accuracy(name="accuracy")]

In [ ]:
#opt = optimizers.Nadam()

In [ ]:
NN_Model.compile(loss='binary_crossentropy', optimizer="adam", metrics=metrics_)
#tf.keras.metrics.Precision(), tf.keras.metrics.Recall() tf.keras.metrics.AUC()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
class_weight = {0: 1/(y==0).sum(), 1: 1/(y==1).sum()}

In [ ]:
scaler = StandardScaler()

In [ ]:
feats = X.columns
X = scaler.fit_transform(X)

X_valid = scaler.transform(valid[feats])
y_valid = valid['TARGET']
X_test_final = scaler.transform(test_final[feats])
y_test_final = test_final['TARGET']

In [ ]:
NN_Model.fit(x = X, y = y, epochs = 128, batch_size = 2048, class_weight = class_weight)#, callbacks = [callback])


Epoch 1/128
200000/200000 [==============================] - 23s 114us/step - loss: 7.4250e-06 - fn: 8423.0000 - fp: 83530.0000 - tn: 100312.0000 - tp: 7735.0000 - precision: 0.0848 - recall: 0.4787 - AUC: 0.5170 - accuracy: 1.3500e-04
Epoch 2/128
200000/200000 [==============================] - 13s 65us/step - loss: 6.9989e-06 - fn: 6012.0000 - fp: 97129.0000 - tn: 86713.0000 - tp: 10146.0000 - precision: 0.0946 - recall: 0.6279 - AUC: 0.5686 - accuracy: 5.0000e-05- loss: 6.9777e-06 - fn: 587.0000 - fp: 10408.0000 - tn: 84
Epoch 3/128
200000/200000 [==============================] - 14s 70us/step - loss: 6.8048e-06 - fn: 5702.0000 - fp: 86722.0000 - tn: 97120.0000 - tp: 10456.0000 - precision: 0.1076 - recall: 0.6471 - AUC: 0.6193 - accuracy: 5.5000e-053s - loss: 6.8266e-06 - fn: 4239.0000 - fp: 64670.0000 - tn: 70907.0000 - tp: 7640.0000 - precision: 0.1057 - recall: 0.6432 - AUC:
Epoch 4/128
200000/200000 [==============================] - 14s 71us/step - loss: 6.6020e-06 - fn: 537

C:\Anaconda3\envs\benedito_deep\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (1.531205). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
C:\Anaconda3\envs\benedito_deep\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.773414). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


200000/200000 [==============================] - 14s 69us/step - loss: 5.9167e-06 - fn: 4929.0000 - fp: 55863.0000 - tn: 127979.0000 - tp: 11229.0000 - precision: 0.1674 - recall: 0.6949 - AUC: 0.7558 - accuracy: 1.5000e-05 - loss: 5.8731e-06 - fn: 714.0000 - fp: 8035.0000 - tn: 18317.0000 - tp: 1606.0000 - precision: 0.1666 - recall: 0.6922 - AUC: 0.7609 - accuracy: 3.48 - ETA: 27s - loss: 5.9029e-06 - fn: 866.0000 - fp
Epoch 16/128
200000/200000 [==============================] - 80040s 400ms/step - loss: 5.8669e-06 - fn: 4873.0000 - fp: 55429.0000 - tn: 128413.0000 - tp: 11285.0000 - precision: 0.1692 - recall: 0.6984 - AUC: 0.7604 - accuracy: 1.0000e-05
Epoch 17/128
200000/200000 [==============================] - 8s 41us/step - loss: 5.8707e-06 - fn: 4790.0000 - fp: 55342.0000 - tn: 128500.0000 - tp: 11368.0000 - precision: 0.1704 - recall: 0.7036 - AUC: 0.7611 - accuracy: 2.0000e-054s - loss: 5.9196e-06 - fn: 2242.0000 - fp: 24929.0000 - tn: 57827.0000 - tp: 5114.0000 - precisi
E

200000/200000 [==============================] - 10s 51us/step - loss: 5.5915e-06 - fn: 4456.0000 - fp: 53151.0000 - tn: 130691.0000 - tp: 11702.0000 - precision: 0.1804 - recall: 0.7242 - AUC: 0.7862 - accuracy: 3.5000e-05
Epoch 70/128
200000/200000 [==============================] - 10s 50us/step - loss: 5.6029e-06 - fn: 4426.0000 - fp: 53199.0000 - tn: 130643.0000 - tp: 11732.0000 - precision: 0.1807 - recall: 0.7261 - AUC: 0.7853 - accuracy: 3.5000e-05
Epoch 71/128
200000/200000 [==============================] - 10s 49us/step - loss: 5.5802e-06 - fn: 4387.0000 - fp: 53164.0000 - tn: 130678.0000 - tp: 11771.0000 - precision: 0.1813 - recall: 0.7285 - AUC: 0.7873 - accuracy: 5.0000e-05
Epoch 72/128
200000/200000 [==============================] - 10s 50us/step - loss: 5.5921e-06 - fn: 4420.0000 - fp: 53308.0000 - tn: 130534.0000 - tp: 11738.0000 - precision: 0.1805 - recall: 0.7265 - AUC: 0.7861 - accuracy: 3.0000e-05s - loss: 5.6024e-06 - fn: 3634.0000 - fp: 43566.0000 - tn: 106985

200000/200000 [==============================] - 10s 52us/step - loss: 5.4622e-06 - fn: 4200.0000 - fp: 53550.0000 - tn: 130292.0000 - tp: 11958.0000 - precision: 0.1825 - recall: 0.7401 - AUC: 0.7964 - accuracy: 9.0000e-05
Epoch 125/128
200000/200000 [==============================] - 10s 51us/step - loss: 5.4672e-06 - fn: 4123.0000 - fp: 54180.0000 - tn: 129662.0000 - tp: 12035.0000 - precision: 0.1818 - recall: 0.7448 - AUC: 0.7960 - accuracy: 1.2500e-04
Epoch 126/128
200000/200000 [==============================] - 12s 58us/step - loss: 5.4627e-06 - fn: 4114.0000 - fp: 54144.0000 - tn: 129698.0000 - tp: 12044.0000 - precision: 0.1820 - recall: 0.7454 - AUC: 0.7961 - accuracy: 8.5000e-05
Epoch 127/128
200000/200000 [==============================] - 10s 51us/step - loss: 5.4688e-06 - fn: 4152.0000 - fp: 53812.0000 - tn: 130030.0000 - tp: 12006.0000 - precision: 0.1824 - recall: 0.7430 - AUC: 0.7960 - accuracy: 1.2500e-04
Epoch 128/128
200000/200000 [==============================] -

In [ ]:
predictions = NN_Model.predict(X_valid)
predictions_ = (predictions > 0.5).astype(int)

In [ ]:
acc_valid = metrics_sk.accuracy_score(y_valid, (NN_Model.predict(X_valid)>0.5))
f1_valid = metrics_sk.f1_score(y_valid, (NN_Model.predict(X_valid)>0.5))
precision_valid = metrics_sk.precision_score(y_valid, (NN_Model.predict(X_valid)>0.5))
recall_valid = metrics_sk.recall_score(y_valid, (NN_Model.predict(X_valid)>0.5))
auc_valid = metrics_sk.roc_auc_score(y_valid, NN_Model.predict(X_valid))

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.70499
precision_valid:  0.173498977442691
recall_valid:  0.7071295722256664
F1_valid:  0.27797506796306726
AUC_valid:  0.769899829831388


## LightGBM

In [ ]:
#dtree = LGBMClassifier(max_depth = 2, min_samples_split=100, min_data_in_leaf = 250,n_estimators=300,
#                            min_impurity_decrease=1e-03, lambda_l1=0.3,
#                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [ ]:
###GRID SEARCH LGBM ---> NAO RODAR SEMPRE

list_i = [2, 4, 6]
list_j = [50, 100, 200]
list_k = [200, 300, 400]
list_l = [100, 200, 300]

for i in list_i:
    for j in list_j:
        for k in list_k:
            for l in list_l:
                dtree = LGBMClassifier(max_depth = i, min_samples_split=j, min_data_in_leaf = k,n_estimators=l,
                            min_impurity_decrease=1e-03, lambda_l1=0.3, lambda_l2=0.7, random_state=11235, eta = 0.005)
                class_tree = dtree.fit(X_train, y_train)
                auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])
                print('depth: ', i, 'samples_split: ', j, 'data_in_leaf: ', k, 'n_estimators: ', l, 'auc_valid: ', auc_valid)

In [ ]:
dtree = LGBMClassifier(max_depth = 4, min_samples_split=50, min_data_in_leaf = 400,n_estimators=300,
                            min_impurity_decrease=1e-03, lambda_l1=0.3,
                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [ ]:
class_tree = dtree.fit(X, y)

In [ ]:
acc_valid = metrics_sk.accuracy_score(y_valid, class_tree.predict(X_valid))
f1_valid = metrics_sk.f1_score(y_valid, class_tree.predict(X_valid))
precision_valid = metrics_sk.precision_score(y_valid, class_tree.predict(X_valid))
recall_valid = metrics_sk.recall_score(y_valid, class_tree.predict(X_valid))
auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.92035
precision_valid:  0.5793650793650794
recall_valid:  0.04525728456292622
F1_valid:  0.08395629672225416
AUC_valid:  0.7918643091019152


## Stacking NN+LGBM

In [ ]:
pred_nn = NN_Model.predict(X_valid)
pred_lgbm = class_tree.predict_proba(X_valid)[:,1].reshape(-1, 1)


In [ ]:
X_stack = np.zeros((pred_nn.shape[0], 2))

In [ ]:
X_stack[:,0] = pred_nn.reshape(1, -1)
X_stack[:,1] = pred_lgbm.reshape(1, -1)
#X_stack[:,2] = pred_nn.reshape(1, -1)**2
#X_stack[:,3] = pred_lgbm.reshape(1, -1)**2
#X_stack[:,4] = pred_nn.reshape(1, -1)**3
#X_stack[:,5] = pred_lgbm.reshape(1, -1)**3

In [ ]:
class_weight = {0: 1/(y_valid==0).sum(), 1: 1/(y_valid==1).sum()}

In [ ]:
lr = LogisticRegression()#class_weight=class_weight

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB(var_smoothing = 0.01)

In [ ]:
lr_ = lr.fit(X_stack, y_valid)

In [ ]:
nb_ = nb.fit(X_stack, y_valid)

## Validação - supervisor logistico

In [ ]:
acc_stack = metrics_sk.accuracy_score(y_valid, lr_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, lr_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, lr_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, lr_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, lr_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.92032
precision_stack:  0.546234509056244
recall_stack:  0.07104773713577185
F1_stack:  0.12574061882817644
AUC_stack:  0.7841383675243476


## Validação - supervisor Naive Bayes

In [ ]:
acc_stack = metrics_sk.accuracy_score(y_valid, nb_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, nb_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, nb_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, nb_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, nb_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.89621
precision_stack:  0.33863319386331936
recall_stack:  0.30105393676379416
F1_stack:  0.31873974401050215
AUC_stack:  0.7804377179475067


In [ ]:
pred_nn_final = NN_Model.predict(X_test_final)
pred_lgbm_final = class_tree.predict_proba(X_test_final)[:,1].reshape(-1, 1)

In [ ]:
X_stack_final = np.zeros((pred_nn_final.shape[0], 2))

In [ ]:
X_stack_final[:,0] = pred_nn_final.reshape(1, -1)
X_stack_final[:,1] = pred_lgbm_final.reshape(1, -1)
#X_stack_final[:,2] = pred_nn_final.reshape(1, -1)**2
#X_stack_final[:,3] = pred_lgbm_final.reshape(1, -1)**2
#X_stack_final[:,4] = pred_nn_final.reshape(1, -1)**3
#X_stack_final[:,5] = pred_lgbm_final.reshape(1, -1)**3

## Teste final - supervisor Naive Bayes

In [ ]:
acc_stack = metrics_sk.accuracy_score(y_test_final, nb_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, nb_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, nb_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, nb_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, nb_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.8920250299560645
precision_stack:  0.30895795246800734
recall_stack:  0.2807308970099668
F1_stack:  0.29416884247171454
AUC_stack:  0.7841536558074138


## Teste final - supervisor logístico

In [ ]:
acc_stack = metrics_sk.accuracy_score(y_test_final, lr_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, lr_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, lr_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, lr_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, lr_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.9194514711756091
precision_stack:  0.4819277108433735
recall_stack:  0.0664451827242525
F1_stack:  0.11678832116788322
AUC_stack:  0.7867029330994432


# Optimizing F1 Score

In [ ]:
#### optimizing f1 score
X_opt = np.zeros((X_stack.shape[0], 4))

In [ ]:
X_opt[:,0] = 1
X_opt[:,1:3]=X_stack
X_opt[:,3] = y_valid

In [ ]:
#def sigmoid(x): resulta em overflow essa merda
#    return 1/(1+np.exp(-x))

In [ ]:
def sigmoid(x): ##corrigido
    if x.all()>=0:
        z = np.exp(-x)
        return 1/(1+z)
    else:
        z = np.exp(-x)
        return z/(1+z)

In [ ]:
def sigmoid1(x): ##derivada
    return sigmoid(x)*(1-sigmoid(x))

In [ ]:
alpha = 0.25 ##obtive via grid search 
gamma = 2

In [ ]:
def f(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    F = A/(alpha*npos +(1-alpha)*mpos)
    return -F

In [ ]:
def logf(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    logF = np.log(A) - np.log((alpha*npos +(1-alpha)*mpos))
    return -logF.round(3)

In [ ]:
def df(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    h = 1/(alpha*npos + (1-alpha)*mpos)
    dF=np.zeros((theta.shape))
    dA = np.zeros((theta.shape))
    dmpos = np.zeros((theta.shape))
    
    for j in range(X_opt.shape[1]-1):
        dA[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta))*gamma*X_opt[X_opt[:,3]==1,j])
        dmpos[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[:,:3], theta))*gamma*X_opt[:,j])
        dF[j] = h*dA[j]-(h**2)*A*(1-alpha)*dmpos[j]
    return -dF
        
    

In [ ]:
def dlogf(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    dlogF=np.zeros((theta.shape))
    dA = np.zeros((theta.shape))
    dmpos = np.zeros((theta.shape))
    
    for j in range(X_opt.shape[1]-1):
        dA[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta))*gamma*X_opt[X_opt[:,3]==1,j])
        dmpos[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[:,:3], theta))*gamma*X_opt[:,j])
        dlogF[j] = (1/A)*dA[j] - (1/(alpha*npos + (1-alpha)*mpos))*(1-alpha)*dmpos[j]
    return -dlogF.round(3)

In [ ]:
from scipy import optimize

In [ ]:
def predict_opt(X, theta_opt):
    score = sigmoid(np.dot(X[:,:-1], theta_opt))
    predict = (score>0.5).astype(int)
    return (score, predict)

In [ ]:
##GRID SEARCH + RANDOM INITIALIZATION --> nao rodar sempre
np.random.seed(42)
theta_matrix = np.random.normal(loc=0.0, scale=20.0, size=(150, 3)).round(3)

list_alpha = [0.10, 0.25, 0.5, 0.75] 
list_gamma = [2, 4, 8, 16, 32]

lista_alpha = []
lista_gamma = []
lista_theta = []
lista_theta_opt = []
lista_auc = []
lista_precision = []
lista_recall = []
lista_f1 = []

for i in range(len(list_alpha)):
    for j in range(len(list_gamma)):
        for k in range(theta_matrix.shape[0]):
            
            alpha = list_alpha[i]
            gamma = list_gamma[j]
            theta = theta_matrix[k,:].T
            theta_opt = optimize.minimize(logf, theta, jac = dlogf, method = 'bfgs').x
            #theta_opt,k = bfgs_method(f, df, theta, maxiter = None, epsi = 0.0001)
            score, predicted = predict_opt(X_opt, theta_opt)
            print('alpha = ', alpha, 'gamma = ', gamma, 'theta_init = ', theta,
                  'theta_opt = ', theta_opt,
                  'AUC = ', metrics_sk.roc_auc_score(y_valid, score),
                  'precision = ', metrics_sk.precision_score(y_valid, predicted),
                  'recall = ', metrics_sk.recall_score(y_valid, predicted),
                  'F1 = ', metrics_sk.f1_score(y_valid, predicted))
            lista_alpha.append(alpha)
            lista_gamma.append(gamma)
            lista_theta.append(theta)
            lista_theta_opt.append(theta_opt)
            lista_auc.append(metrics_sk.roc_auc_score(y_valid, score))
            lista_precision.append(metrics_sk.precision_score(y_valid, predicted))
            lista_recall.append(metrics_sk.recall_score(y_valid, predicted))
            lista_f1.append(metrics_sk.f1_score(y_valid, predicted))

alpha =  0.1 gamma =  2 theta_init =  [ 9.934 -2.765 12.954] theta_opt =  [ 9.934 -2.765 12.954] AUC =  0.6869658342063625 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [30.461 -4.683 -4.683] theta_opt =  [30.461 -4.683 -4.683] AUC =  0.22088419771226409 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [31.584 15.349 -9.389] theta_opt =  [31.584 15.349 -9.389] AUC =  0.6888694474056797 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [10.851 -9.268 -9.315] theta_opt =  [14.03395897 -6.59244982 -8.24641657] AUC =  0.21941897478645978 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [  4.839 -38.266 -34.498] theta_opt =  [ 63.20747313 -32.10452724 -31.80071188] AUC =  0.3167170981977988 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [-11

alpha =  0.1 gamma =  2 theta_init =  [-23.374  22.856  15.039] theta_opt =  [-22.41757084  23.60577511  15.40639437] AUC =  0.7764936297110909 precision =  0.33742926434923204 recall =  0.2587724736515809 F1 =  0.2929122807017544
alpha =  0.1 gamma =  2 theta_init =  [ 15.821 -18.188  28.056] theta_opt =  [ 15.821 -18.188  28.056] AUC =  0.3830475081268333 precision =  0.08064725825324279 recall =  0.9998760074395536 F1 =  0.14925594136373735
alpha =  0.1 gamma =  2 theta_init =  [-28.037  11.737  43.809] theta_opt =  [-23.8931045   15.22621667  45.48562044] AUC =  0.7862655112504855 precision =  0.3669792241220813 recall =  0.24748915065096094 F1 =  0.29561611374407587
alpha =  0.1 gamma =  2 theta_init =  [-19.811 -11.326   1.993] theta_opt =  [15.81087868 -9.09506569  5.88573436] AUC =  0.26310806736922376 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [-10.07  -31.013   1.371] theta_opt =  [ 28.50618459 -29.88173803   7.84751437]

C:\Anaconda3\envs\benedito_deep\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


alpha =  0.1 gamma =  2 theta_init =  [-14.306  13.592 -14.607] theta_opt =  [-1478.86514975  1160.68626407  2112.1658269 ] AUC =  0.7477014546147408 precision =  0.36740488358886997 recall =  0.24066955982641042 F1 =  0.29083008690440515
alpha =  0.1 gamma =  2 theta_init =  [  4.329   0.911 -13.032] theta_opt =  [  7.2025711    3.25574788 -11.93050963] AUC =  0.36944887508631247 precision =  0.08063161263225264 recall =  0.9997520148791073 F1 =  0.1492277652850652
alpha =  0.1 gamma =  2 theta_init =  [ 42.879  12.678 -40.503] theta_opt =  [ 42.879  12.678 -40.503] AUC =  0.4571624733788068 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [  3.729 -13.236  17.049] theta_opt =  [  7.57703273 -10.25753479  18.35614646] AUC =  0.4230864099750251 precision =  0.08066904085470769 recall =  0.9998760074395536 F1 =  0.14929324533227187
alpha =  0.1 gamma =  2 theta_init =  [-15.85   -2.295  10.1  ] theta_opt =  [-8.54354593  5.33434014 16.80

alpha =  0.1 gamma =  2 theta_init =  [  4.641 -28.962 -28.149] theta_opt =  [ 50.42539181 -23.0477612  -25.79867832] AUC =  0.2517163130869134 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [-14.369  -4.269   6.218] theta_opt =  [ 4.47946966  1.73423759 13.87047868] AUC =  0.790808124732726 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [29.507 17.153 -3.199] theta_opt =  [29.507 17.153 -3.199] AUC =  0.7160645737771751 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [ -0.38  -20.051  -0.37 ] theta_opt =  [ 18.6031932  -17.93254045   0.85440347] AUC =  0.23363088648139532 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.1 gamma =  2 theta_init =  [ -5.773   6.454 -16.545] theta_opt =  [-69.58633791  80.05931783  28.37000978] AUC =  0.773372322846902 precision =  0.3250186706497386 recall =  0.2698078115313081 

...

alpha =  0.75 gamma =  32 theta_init =  [-11.246 -20.257   6.285] theta_opt =  [ 15.88468698 -19.66359916  18.89660054] AUC =  0.2950914934879292 precision =  0.08066661998459491 recall =  0.9998760074395536 F1 =  0.14928909952606637
alpha =  0.75 gamma =  32 theta_init =  [-18.16  -28.246  29.313] theta_opt =  [ 23.51238526 -28.23344137  49.80320826] AUC =  0.4190201822893618 precision =  0.0806512911807653 recall =  0.9998760074395536 F1 =  0.14926284809950857
alpha =  0.75 gamma =  32 theta_init =  [ -4.516   1.351 -28.495] theta_opt =  [ 13.55382587   9.39430267 -27.21174606] AUC =  0.4338886955192979 precision =  0.08059644771581727 recall =  0.9992560446373218 F1 =  0.14916201634322626
alpha =  0.75 gamma =  32 theta_init =  [-10.888   2.218 -23.02 ] theta_opt =  [  8.72989926  10.23191185 -22.13719453] AUC =  0.521040822697753 precision =  0.0805980598059806 recall =  0.9992560446373218 F1 =  0.14916477719679794
alpha =  0.75 gamma =  32 theta_init =  [  7.514 -12.013  -5.834] t

alpha =  0.75 gamma =  32 theta_init =  [-10.07  -31.013   1.371] theta_opt =  [ 28.50618459 -29.88173803   7.84751437] AUC =  0.2403953762447936 precision =  0.08064564519516366 recall =  0.9998760074395536 F1 =  0.14925317884839623
alpha =  0.75 gamma =  32 theta_init =  [-21.246   9.472 -18.388] theta_opt =  [-87.91350867  79.8595208   86.3785307 ] AUC =  0.7796202733736883 precision =  0.37468827930174564 recall =  0.22355858648481092 F1 =  0.28003416944940596
alpha =  0.75 gamma =  32 theta_init =  [ 30.999 -15.665  -6.441] theta_opt =  [ 30.999 -15.665  -6.441] AUC =  0.22594290738378833 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [ 16.27  -24.617   4.549] theta_opt =  [ 23.00011912 -19.84010983   6.13246555] AUC =  0.2423324809898473 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [ 26.143 -32.15    3.693] theta_opt =  [ 28.37767846 -30.23811184   4.46824804] AUC =  0.23

C:\Anaconda3\envs\benedito_deep\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


alpha =  0.75 gamma =  32 theta_init =  [-14.306  13.592 -14.607] theta_opt =  [-1478.86514975  1160.68626407  2112.1658269 ] AUC =  0.7477014546147408 precision =  0.36740488358886997 recall =  0.24066955982641042 F1 =  0.29083008690440515
alpha =  0.75 gamma =  32 theta_init =  [  4.329   0.911 -13.032] theta_opt =  [  7.2025711    3.25574788 -11.93050963] AUC =  0.36944887508631247 precision =  0.08063161263225264 recall =  0.9997520148791073 F1 =  0.1492277652850652
alpha =  0.75 gamma =  32 theta_init =  [ 42.879  12.678 -40.503] theta_opt =  [ 42.879  12.678 -40.503] AUC =  0.4571624733788068 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [  3.729 -13.236  17.049] theta_opt =  [  7.57703273 -10.25753479  18.35614646] AUC =  0.4230864099750251 precision =  0.08066904085470769 recall =  0.9998760074395536 F1 =  0.14929324533227187
alpha =  0.75 gamma =  32 theta_init =  [-15.85   -2.295  10.1  ] theta_opt =  [-8.54354593  5.3343

alpha =  0.75 gamma =  32 theta_init =  [  4.641 -28.962 -28.149] theta_opt =  [ 50.42539181 -23.0477612  -25.79867832] AUC =  0.2517163130869134 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [-14.369  -4.269   6.218] theta_opt =  [ 4.47946966  1.73423759 13.87047868] AUC =  0.790808124732726 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [29.507 17.153 -3.199] theta_opt =  [29.507 17.153 -3.199] AUC =  0.7160645737771751 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [ -0.38  -20.051  -0.37 ] theta_opt =  [ 18.6031932  -17.93254045   0.85440347] AUC =  0.23363088648139532 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  32 theta_init =  [ -5.773   6.454 -16.545] theta_opt =  [-69.58633791  80.05931783  28.37000978] AUC =  0.773372322846902 precision =  0.3250186706497386 recall =  0.2698078

In [ ]:
results = pd.DataFrame()
results['alpha']=lista_alpha
results['gamma']=lista_gamma
results['theta']=lista_theta
results['theta_opt']=lista_theta_opt
results['AUC']=lista_auc
results['Precision'] = lista_precision
results['Recall'] = lista_recall
results['F1'] = lista_f1
results.to_csv("resultados_log_otimizacao_table_5.csv")

In [ ]:
#alpha =  0.25 gamma =  2 theta_init =  [-1 -4  9] theta_opt =  [-568.50174834  322.85256065 2247.08499949] AUC =  0.7829109328873998 precision =  0.2642267969778887 recall =  0.4726596404215747 F1 =  0.338964965321003
#alpha =  0.25 gamma =  2 theta_init =  [0 3 5] theta_opt =  [-431.15928105  220.73593698 1731.39766752] AUC =  0.7840137316349044 precision =  0.26989136817381093 recall =  0.45592064476131433 F1 =  0.3390658859329614
#0.1	2	[-12.902   7.228  30.761]	[-12.902   7.228  30.761]	0.7882440904853698	0.34440142942227514	0.28679479231246124	0,312969353
#0.1	2	[-21.154  16.451 -24.417]	[-89.98832355  91.3083205   81.7098475 ]	0.7784056931783964	0.3293559516963772	0.28406695598264103	0,305039611


In [ ]:
alpha = 0.25
gamma = 2
theta_opt = np.array([-431.15928105,  220.73593698, 1731.39766752])

In [ ]:
score, predicted = predict_opt(X_opt, theta_opt)

### Performance em validação (usado para otimizar)

In [ ]:
print('AUC:', metrics_sk.roc_auc_score(y_valid, score))
print('PRECISION: ', metrics_sk.precision_score(y_valid, predicted))
print('RECALL: ', metrics_sk.recall_score(y_valid, predicted))
print('F1: ', metrics_sk.f1_score(y_valid, predicted))

AUC: 0.7860406893452276
PRECISION:  0.2681905041893282
RECALL:  0.4524488530688159
F1:  0.3367634165474597


In [ ]:
X_opt_final = np.zeros((X_stack_final.shape[0], 4))
X_opt_final[:,0] = 1
X_opt_final[:,1:3]=X_stack_final
X_opt_final[:,3] = y_test_final

In [ ]:
score_final, predicted_final = predict_opt(X_opt_final, theta_opt)

In [ ]:
X_opt_final.shape

(7511, 4)

In [ ]:
X_stack_final.shape

(7511, 2)

### Performance no teste final

In [ ]:
print('ACCURACY', metrics_sk.roc_auc_score(y_test_final, predicted_final))
print('AUC', metrics_sk.roc_auc_score(y_test_final, score_final))
print('PRECISION', metrics_sk.precision_score(y_test_final, predicted_final))
print('RECALL', metrics_sk.recall_score(y_test_final, predicted_final))
print('F1', metrics_sk.f1_score(y_test_final, predicted_final))

AUC 0.787555857855972
PRECISION 0.2536443148688047
RECALL 0.4335548172757475
F1 0.3200490496627836


In [ ]:
from eli5.permutation_importance import get_score_importances

In [ ]:
def score(X, y):
    y_pred = NN_Model.predict(X)
    return metrics_sk.roc_auc_score(y, y_pred)

base_score, score_decreases = get_score_importances(score, X_valid, y_valid)
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=feature_importances[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

{'mean_knn_target': 0.08827339310965651, 'EXT_SOURCE_3': 0.0059802959927043545, 'EXT_SOURCE_2': 0.005133701197485507, 'EXT_SOURCE_1': 0.005048921225274739, 'SUM(previous.CNT_PAYMENT)': 0.004377050944137606, 'amt_credit_sum_consumer_credit_bureau': 0.0034124577423380485, 'avg_pos_cash_cnt_installment_future_previous': 0.003362544718193039, 'CODE_GENDER_M': 0.0033236523108880257, 'amt_credit_sum_debt_credit_card_bureau': 0.0031107112221224974, 'cnt_refused_previous': 0.002999685449884093, 'NAME_CONTRACT_TYPE_Revolving loans': 0.0027290211611070035, 'max_pos_cash_sk_dpd_def_previous': 0.0025829880413299256, 'SUM(previous.DAYS_FIRST_DRAWING)': 0.0024231375094488695, 'amt_paid_down': 0.0021969400669918484, 'NAME_EDUCATION_TYPE_Higher education': 0.00216294559173138, 'STD(previous.DAYS_FIRST_DRAWING)': 0.002122711229810137, 'NAME_FAMILY_STATUS_Married': 0.002038711344584265, 'MIN(previous.CNT_PAYMENT)': 0.0020239952949318908, 'cnt_yield_low_normal_previous': 0.0019751716142476948, 'avg_amt_a

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

{'mean_knn_target': 0.08827339310965651,
 'EXT_SOURCE_3': 0.0059802959927043545,
 'EXT_SOURCE_2': 0.005133701197485507,
 'EXT_SOURCE_1': 0.005048921225274739,
 'SUM(previous.CNT_PAYMENT)': 0.004377050944137606,
 'amt_credit_sum_consumer_credit_bureau': 0.0034124577423380485,
 'avg_pos_cash_cnt_installment_future_previous': 0.003362544718193039,
 'CODE_GENDER_M': 0.0033236523108880257,
 'amt_credit_sum_debt_credit_card_bureau': 0.0031107112221224974,
 'cnt_refused_previous': 0.002999685449884093,
 'NAME_CONTRACT_TYPE_Revolving loans': 0.0027290211611070035,
 'max_pos_cash_sk_dpd_def_previous': 0.0025829880413299256,
 'SUM(previous.DAYS_FIRST_DRAWING)': 0.0024231375094488695,
 'amt_paid_down': 0.0021969400669918484,
 'NAME_EDUCATION_TYPE_Higher education': 0.00216294559173138,
 'STD(previous.DAYS_FIRST_DRAWING)': 0.002122711229810137,
 'NAME_FAMILY_STATUS_Married': 0.002038711344584265,
 'MIN(previous.CNT_PAYMENT)': 0.0020239952949318908,
 'cnt_yield_low_normal_previous': 0.0019751716142

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=class_tree.feature_importances_[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

{'mean_knn_target': 155, 'EXT_SOURCE_1': 68, 'avg_pos_cash_cnt_installment_future_previous': 63, 'AGE': 63, 'EXT_SOURCE_2': 50, 'AMT_ANNUITY': 48, 'REGION_POPULATION_RELATIVE': 48, 'EXT_SOURCE_3': 44, 'amt_paid_down': 43, 'DAYS_ID_PUBLISH': 42, 'max_installments_gap_amt_payment_previous': 39, 'delta_avg_pos_cash_cnt_installment_previous': 36, 'loan_tot_lim_ratio': 32, 'MONTHS_EMPLOYED': 32, 'DAYS_EMPLOYED': 30, 'tot_debt_lim_ratio': 30, 'amt_credit_sum_credit_total_bureau': 30, 'MEDIAN(bureau.DAYS_CREDIT_ENDDATE)': 30, 'max_pos_cash_sk_dpd_def_previous': 29, 'amt_credit_sum_consumer_credit_bureau': 29, 'gap_employment_registration': 29, 'OWN_CAR_AGE': 28, 'MEDIAN(bureau.AMT_CREDIT_SUM)': 28, 'avg_cnt_credit_card_drawings_atm_previous': 27, 'days_credit_bureau': 27, 'MAX(bureau.DAYS_CREDIT)': 27, 'days_phone_change_over_gap_amt_installment': 26, 'gap_avg_application_credit_previous': 26, 'days_credit_enddate_bureau': 25, 'remaining_pmts': 25, 'sum_tot_credit_lim': 24, 'credit_income_rat

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

{'mean_knn_target': 155,
 'EXT_SOURCE_1': 68,
 'avg_pos_cash_cnt_installment_future_previous': 63,
 'AGE': 63,
 'EXT_SOURCE_2': 50,
 'AMT_ANNUITY': 48,
 'REGION_POPULATION_RELATIVE': 48,
 'EXT_SOURCE_3': 44,
 'amt_paid_down': 43,
 'DAYS_ID_PUBLISH': 42,
 'max_installments_gap_amt_payment_previous': 39,
 'delta_avg_pos_cash_cnt_installment_previous': 36,
 'loan_tot_lim_ratio': 32,
 'MONTHS_EMPLOYED': 32,
 'DAYS_EMPLOYED': 30,
 'tot_debt_lim_ratio': 30,
 'amt_credit_sum_credit_total_bureau': 30,
 'MEDIAN(bureau.DAYS_CREDIT_ENDDATE)': 30,
 'max_pos_cash_sk_dpd_def_previous': 29,
 'amt_credit_sum_consumer_credit_bureau': 29,
 'gap_employment_registration': 29,
 'OWN_CAR_AGE': 28,
 'MEDIAN(bureau.AMT_CREDIT_SUM)': 28,
 'avg_cnt_credit_card_drawings_atm_previous': 27,
 'days_credit_bureau': 27,
 'MAX(bureau.DAYS_CREDIT)': 27,
 'days_phone_change_over_gap_amt_installment': 26,
 'gap_avg_application_credit_previous': 26,
 'days_credit_enddate_bureau': 25,
 'remaining_pmts': 25,
 'sum_tot_credi

In [ ]:
def score(X, y):
    y_pred = class_tree.predict(X)
    return metrics_sk.roc_auc_score(y, y_pred)

base_score, score_decreases = get_score_importances(score, X_valid, y_valid)
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=feature_importances[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

{'mean_knn_target': 0.01802910294953195, 'EXT_SOURCE_2': 0.005835016930038739, 'EXT_SOURCE_3': 0.0035957640224732355, 'tot_debt_lim_ratio': 0.0026538548438711507, 'EXT_SOURCE_1': 0.002387390535868428, 'max_installments_gap_amt_payment_previous': 0.0014668015769382149, 'STD(previous.CNT_PAYMENT)': 0.0014343930357815448, 'SUM(previous.DAYS_FIRST_DRAWING)': 0.0014080725178787557, 'avg_cnt_credit_card_drawings_atm_previous': 0.0014015461677400864, 'max_pos_cash_sk_dpd_def_previous': 0.00139241413285911, 'FLOORSMIN_MODE': 0.0012882116940825083, 'FLOORSMAX_AVG': 0.001278856449664878, 'avg_pos_cash_cnt_installment_future_previous': 0.0012331760447883643, 'amt_credit_sum_consumer_credit_bureau': 0.0012294826350229826, 'MEDIAN(bureau.DAYS_CREDIT_UPDATE)': 0.0011988071709334136, 'STD(bureau.DAYS_ENDDATE_FACT)': 0.0011131001036440934, 'MEDIAN(bureau.DAYS_CREDIT_ENDDATE)': 0.000990195268220817, 'delta_avg_pos_cash_cnt_installment_previous': 0.0009871512565938323, 'MONTHS_EMPLOYED': 0.0009245015321

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

{'mean_knn_target': 0.01802910294953195,
 'EXT_SOURCE_2': 0.005835016930038739,
 'EXT_SOURCE_3': 0.0035957640224732355,
 'tot_debt_lim_ratio': 0.0026538548438711507,
 'EXT_SOURCE_1': 0.002387390535868428,
 'max_installments_gap_amt_payment_previous': 0.0014668015769382149,
 'STD(previous.CNT_PAYMENT)': 0.0014343930357815448,
 'SUM(previous.DAYS_FIRST_DRAWING)': 0.0014080725178787557,
 'avg_cnt_credit_card_drawings_atm_previous': 0.0014015461677400864,
 'max_pos_cash_sk_dpd_def_previous': 0.00139241413285911,
 'FLOORSMIN_MODE': 0.0012882116940825083,
 'FLOORSMAX_AVG': 0.001278856449664878,
 'avg_pos_cash_cnt_installment_future_previous': 0.0012331760447883643,
 'amt_credit_sum_consumer_credit_bureau': 0.0012294826350229826,
 'MEDIAN(bureau.DAYS_CREDIT_UPDATE)': 0.0011988071709334136,
 'STD(bureau.DAYS_ENDDATE_FACT)': 0.0011131001036440934,
 'MEDIAN(bureau.DAYS_CREDIT_ENDDATE)': 0.000990195268220817,
 'delta_avg_pos_cash_cnt_installment_previous': 0.0009871512565938323,
 'MONTHS_EMPLOYED

In [ ]:
feat_selected = ["mean_knn_target",
"EXT_SOURCE_3",
"EXT_SOURCE_2",
"EXT_SOURCE_1",
"SUM(previous.CNT_PAYMENT)",
"amt_credit_sum_consumer_credit_bureau",
"avg_pos_cash_cnt_installment_future_previous",
"CODE_GENDER_M",
"amt_credit_sum_debt_credit_card_bureau",
"cnt_refused_previous",
"NAME_CONTRACT_TYPE_Revolving loans",
"max_pos_cash_sk_dpd_def_previous",
"SUM(previous.DAYS_FIRST_DRAWING)",
"amt_paid_down",
"NAME_EDUCATION_TYPE_Higher education",
"STD(previous.DAYS_FIRST_DRAWING)",
"NAME_FAMILY_STATUS_Married",
"MIN(previous.CNT_PAYMENT)",
"cnt_yield_low_normal_previous",
"avg_amt_application_previous",
"credit_income_ratio",
"remaining_pmts",
"OCCUPATION_TYPE_Sales staff",
"MAX(previous.NFLAG_LAST_APPL_IN_DAY)",
"FLAG_EMAIL",
"missing_rate",
"REGION_RATING_CLIENT_W_CITY_2",
"COUNT(bureau)",
"gap_avg_application_credit_previous",
"days_phone_change_over_gap_amt_installment",
"FLAG_OWN_CAR_Y",
"avg_pos_cash_cnt_installment_previous",
"NAME_EDUCATION_TYPE_Secondary / secondary special",
"YEARS_BUILD_MODE",
"UI_credit_card_bureau",
"ENTRANCES_AVG",
"num_loans_l1m",
"STD(bureau.CREDIT_DAY_OVERDUE)",
"STD(previous.AMT_GOODS_PRICE)",
"max_interest_rate_privileged_previous",
"cnt_contracts_bureau",
"SKEW(previous.DAYS_TERMINATION)",
"MEAN(previous.DAYS_LAST_DUE_1ST_VERSION)",
"MIN(previous.AMT_APPLICATION)",
"MIN(bureau.DAYS_CREDIT)",
"delinquency_6_mths_consumer_credit_bureau",
"MAX(previous.DAYS_FIRST_DRAWING)",
"NAME_EDUCATION_TYPE_Lower secondary",
"MIN(previous.HOUR_APPR_PROCESS_START)",
"avg_amt_credit_card_receivable_principal_previous",
"MEAN(bureau.AMT_CREDIT_SUM)",
"amt_credit_max_overdue_bureau",
"DAYS_ID_PUBLISH",
"tot_debt_lim_ratio",
"avg_amt_credit_card_total_receivable_previous",
"max_interest_rate_primary_previous",
"MEDIAN(previous.NFLAG_INSURED_ON_APPROVAL)",
"num_loans_l6m",
"STD(bureau.DAYS_CREDIT)",
"MEAN(previous.RATE_INTEREST_PRIMARY)",
"FLOORSMIN_AVG",
"SUM(bureau.AMT_CREDIT_SUM_DEBT)",
"avg_cnt_credit_card_drawings_atm_previous",
"APARTMENTS_MEDI",
"LIVINGAREA_MODE",
"delinquency_12_24_mths_microloan_bureau",
"SKEW(bureau.DAYS_CREDIT)",
"LANDAREA_AVG",
"num_loans_l3m",
"FLAG_DOCUMENT_13",
"MAX(bureau.AMT_CREDIT_SUM_DEBT)",
"MAX(previous.CNT_PAYMENT)",
"delinquency_6_12_mths_credit_card_bureau",
"MAX(bureau.AMT_CREDIT_SUM_LIMIT)",
"MEAN(previous.AMT_CREDIT)",
"MAX(bureau.DAYS_CREDIT_UPDATE)",
"WALLSMATERIAL_MODE_Monolithic",
"avg_amt_credit_card_drawings_atm_previous",
"MEDIAN(previous.CNT_PAYMENT)",
"avg_cnt_credit_card_instalment_mature_cum_previous",
"FLAG_PHONE",
"YEARS_BEGINEXPLUATATION_AVG",
"SUM(previous.DAYS_TERMINATION)",
"income_over_credit",
"days_phone_change_times_gap_days_installment",
"SUM(previous.AMT_GOODS_PRICE)",
"LANDAREA_MEDI",
"max_cnt_payment_previous",
"SUM(bureau.AMT_CREDIT_SUM_OVERDUE)",
"MEAN(previous.NFLAG_LAST_APPL_IN_DAY)",
"MIN(previous.RATE_DOWN_PAYMENT)",
"SKEW(bureau.AMT_CREDIT_SUM_LIMIT)",
"SUM(previous.RATE_DOWN_PAYMENT)",
"cnt_portfolio_cash_previous",
"MEDIAN(bureau.CNT_CREDIT_PROLONG)",
"SKEW(previous.DAYS_DECISION)",
"NAME_INCOME_TYPE_State servant",
"STD(previous.AMT_CREDIT)",
"MEAN(bureau.DAYS_ENDDATE_FACT)",
"NAME_TYPE_SUITE_Other_A",
"FLOORSMIN_MEDI",
"DEF_30_CNT_SOCIAL_CIRCLE",
"REG_REGION_NOT_WORK_REGION",
"SKEW(bureau.AMT_CREDIT_MAX_OVERDUE)",
"NONLIVINGAREA_MEDI",
"delinquency_6_12_mths_microloan_bureau",
"avg_amt_credit_card_recivable_previous",
"delinquency_long_total_bureau",
"FLAG_WORK_PHONE",
"cnt_channel_contact_center_previous",
"SUM(previous.DAYS_FIRST_DUE)",
"OWN_CAR_AGE",
"AMT_GOODS_PRICE",
"cnt_appl_previous",
"delinquency_12_24_mths_consumer_credit_bureau",
"cnt_cash_previous",
"NAME_TYPE_SUITE_Family",
"MEAN(previous.AMT_DOWN_PAYMENT)",
"FLAG_DOCUMENT_3",
"NAME_TYPE_SUITE_Group of people",
"cnt_seller_furniture_previous",
"delinquency_6_12_mths_total_bureau",
"CODE_GENDER_XNA",
"amt_credit_sum_working_capital_bureau",
"MEDIAN(previous.RATE_DOWN_PAYMENT)",
"DAYS_REGISTRATION",
"DEF_60_CNT_SOCIAL_CIRCLE",
"SUM(bureau.DAYS_CREDIT_ENDDATE)",
"MIN(bureau.DAYS_CREDIT_ENDDATE)",
"delinquency_24_36_mths_consumer_credit_bureau",
"MAX(previous.AMT_DOWN_PAYMENT)",
"UI_credit_sum_total_bureau",
"SKEW(bureau.DAYS_ENDDATE_FACT)",
"SKEW(previous.AMT_DOWN_PAYMENT)",
"NAME_INCOME_TYPE_Student",
"MEDIAN(bureau.DAYS_CREDIT_UPDATE)",
"credit_income_cluster_4",
"YEARS_BEGINEXPLUATATION_MEDI",
"NAME_HOUSING_TYPE_Office apartment",
"max_credit_card_sk_dpd_def_previous",
"SKEW(previous.AMT_APPLICATION)",
"cnt_product_xsell_previous",
"WALLSMATERIAL_MODE_Stone, brick",
"cnt_yield_low_action_previous",
"MAX(previous.DAYS_DECISION)",
"amt_credit_sum_overdue_microloan_bureau",
"MEAN(previous.DAYS_TERMINATION)",
"OBS_30_CNT_SOCIAL_CIRCLE",
"cnt_seller_electronics_previous",
"MAX(previous.HOUR_APPR_PROCESS_START)",
"WALLSMATERIAL_MODE_Panel",
"MEDIAN(previous.AMT_APPLICATION)",
"down_payment_times_cnt_refused_previous",
"MEDIAN(previous.AMT_CREDIT)",
"max_installments_gap_amt_payment_previous",
"STD(previous.CNT_PAYMENT)",
"FLOORSMIN_MODE",
"FLOORSMAX_AVG",
"STD(bureau.DAYS_ENDDATE_FACT)"]

In [ ]:
data[feat_selected].head()

,mean_knn_target,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,SUM(previous.CNT_PAYMENT),amt_credit_sum_consumer_credit_bureau,avg_pos_cash_cnt_installment_future_previous,CODE_GENDER_M,amt_credit_sum_debt_credit_card_bureau,cnt_refused_previous,NAME_CONTRACT_TYPE_Revolving loans,max_pos_cash_sk_dpd_def_previous,SUM(previous.DAYS_FIRST_DRAWING),amt_paid_down,NAME_EDUCATION_TYPE_Higher education,STD(previous.DAYS_FIRST_DRAWING),NAME_FAMILY_STATUS_Married,MIN(previous.CNT_PAYMENT),cnt_yield_low_normal_previous,avg_amt_application_previous,credit_income_ratio,remaining_pmts,OCCUPATION_TYPE_Sales staff,MAX(previous.NFLAG_LAST_APPL_IN_DAY),FLAG_EMAIL,missing_rate,REGION_RATING_CLIENT_W_CITY_2,COUNT(bureau),gap_avg_application_credit_previous,days_phone_change_over_gap_amt_installment,FLAG_OWN_CAR_Y,avg_pos_cash_cnt_installment_previous,NAME_EDUCATION_TYPE_Secondary / secondary special,YEARS_BUILD_MODE,UI_credit_card_bureau,ENTRANCES_AVG,num_loans_l1m,STD(bureau.CREDIT_DAY_OVERDUE),STD(previous.AMT_GOODS_PRICE),max_interest_rate_privileged_previous,cnt_contracts_bureau,SKEW(previous.DAYS_TERMINATION),MEAN(previous.DAYS_LAST_DUE_1ST_VERSION),MIN(previous.AMT_APPLICATION),MIN(bureau.DAYS_CREDIT),delinquency_6_mths_consumer_credit_bureau,MAX(previous.DAYS_FIRST_DRAWING),NAME_EDUCATION_TYPE_Lower secondary,MIN(previous.HOUR_APPR_PROCESS_START),avg_amt_credit_card_receivable_principal_previous,MEAN(bureau.AMT_CREDIT_SUM),amt_credit_max_overdue_bureau,DAYS_ID_PUBLISH,tot_debt_lim_ratio,avg_amt_credit_card_total_receivable_previous,max_interest_rate_primary_previous,MEDIAN(previous.NFLAG_INSURED_ON_APPROVAL),num_loans_l6m,STD(bureau.DAYS_CREDIT),MEAN(previous.RATE_INTEREST_PRIMARY),FLOORSMIN_AVG,SUM(bureau.AMT_CREDIT_SUM_DEBT),avg_cnt_credit_card_drawings_atm_previous,APARTMENTS_MEDI,LIVINGAREA_MODE,delinquency_12_24_mths_microloan_bureau,SKEW(bureau.DAYS_CREDIT),LANDAREA_AVG,num_loans_l3m,FLAG_DOCUMENT_13,MAX(bureau.AMT_CREDIT_SUM_DEBT),MAX(previous.CNT_PAYMENT),delinquency_6_12_mths_credit_card_bureau,MAX(bureau.AMT_CREDIT_SUM_LIMIT),MEAN(previous.AMT_CREDIT),MAX(bureau.DAYS_CREDIT_UPDATE),WALLSMATERIAL_MODE_Monolithic,avg_amt_credit_card_drawings_atm_previous,MEDIAN(previous.CNT_PAYMENT),avg_cnt_credit_card_instalment_mature_cum_previous,FLAG_PHONE,YEARS_BEGINEXPLUATATION_AVG,SUM(previous.DAYS_TERMINATION),income_over_credit,days_phone_change_times_gap_days_installment,SUM(previous.AMT_GOODS_PRICE),LANDAREA_MEDI,max_cnt_payment_previous,SUM(bureau.AMT_CREDIT_SUM_OVERDUE),MEAN(previous.NFLAG_LAST_APPL_IN_DAY),MIN(previous.RATE_DOWN_PAYMENT),SKEW(bureau.AMT_CREDIT_SUM_LIMIT),SUM(previous.RATE_DOWN_PAYMENT),cnt_portfolio_cash_previous,MEDIAN(bureau.CNT_CREDIT_PROLONG),SKEW(previous.DAYS_DECISION),NAME_INCOME_TYPE_State servant,STD(previous.AMT_CREDIT),MEAN(bureau.DAYS_ENDDATE_FACT),NAME_TYPE_SUITE_Other_A,FLOORSMIN_MEDI,DEF_30_CNT_SOCIAL_CIRCLE,REG_REGION_NOT_WORK_REGION,SKEW(bureau.AMT_CREDIT_MAX_OVERDUE),NONLIVINGAREA_MEDI,delinquency_6_12_mths_microloan_bureau,avg_amt_credit_card_recivable_previous,delinquency_long_total_bureau,FLAG_WORK_PHONE,cnt_channel_contact_center_previous,SUM(previous.DAYS_FIRST_DUE),OWN_CAR_AGE,AMT_GOODS_PRICE,cnt_appl_previous,delinquency_12_24_mths_consumer_credit_bureau,cnt_cash_previous,NAME_TYPE_SUITE_Family,MEAN(previous.AMT_DOWN_PAYMENT),FLAG_DOCUMENT_3,NAME_TYPE_SUITE_Group of people,cnt_seller_furniture_previous,delinquency_6_12_mths_total_bureau,CODE_GENDER_XNA,amt_credit_sum_working_capital_bureau,MEDIAN(previous.RATE_DOWN_PAYMENT),DAYS_REGISTRATION,DEF_60_CNT_SOCIAL_CIRCLE,SUM(bureau.DAYS_CREDIT_ENDDATE),MIN(bureau.DAYS_CREDIT_ENDDATE),delinquency_24_36_mths_consumer_credit_bureau,MAX(previous.AMT_DOWN_PAYMENT),UI_credit_sum_total_bureau,SKEW(bureau.DAYS_ENDDATE_FACT),SKEW(previous.AMT_DOWN_PAYMENT),NAME_INCOME_TYPE_Student,MEDIAN(bureau.DAYS_CREDIT_UPDATE),credit_income_cluster_4,YEARS_BEGINEXPLUATATION_MEDI,NAME_HOUSING_TYPE_Office apartment,max_credit_card_sk_dpd_def_previous,SKEW(previous.AMT_APPLICATIO

In [ ]:
data.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_start_dawn_previous,cnt